# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [2]:
import fsspec
from redis_block_fsspec_cache import RedisCachingFileSystem

fs = fsspec.filesystem(
    "s3",
    anon=True,
)

cached_fs = RedisCachingFileSystem(
    redis_host="localhost",
    redis_port=6380,
    expiry_time=60,
    fs=fs,
)

Let try to access a single chunk from a remote GRIB file we know about in an s3 bucket:

```json
"gust\/0.0.0": [
    "s3:\/\/noaa-hrrr-bdp-pds\/hrrr.20230927\/conus\/hrrr.t00z.wrfsubhf00.grib2",
    2884409,
    1133540
],
```

In [5]:
%%time

nocache_chunk = fs.read_block("s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2", offset=2884409, length=1133540)

CPU times: user 52.5 ms, sys: 21.3 ms, total: 73.8 ms
Wall time: 1.11 s


In [6]:
%%time

uncached_chunk = cached_fs.read_block("s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2", offset=2884409, length=1133540)

CPU times: user 49.5 ms, sys: 22 ms, total: 71.5 ms
Wall time: 1.32 s


In [7]:
%%time

cached_chunk = cached_fs.read_block("s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2", offset=2884409, length=1133540)

CPU times: user 8.55 ms, sys: 8.07 ms, total: 16.6 ms
Wall time: 194 ms


In [15]:
print(len(nocache_chunk))
print(len(uncached_chunk))
print(len(cached_chunk))

1133540
5242880
5242880
